In [14]:
import websocket, json, threading, hashlib, toml
import urllib.parse, hmac, base64, time, requests

def get_kraken_signature(urlpath, data, secret):
    # Encode dat for request into URL.
    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()
    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

def get_ws_token(api_key, api_secret):
    url = "https://api.kraken.com/0/private/GetWebSocketsToken"
    nonce = int(time.time() * 1000)
    data = {
        "nonce": nonce,
    }
    headers = {
        "API-Key": api_key,
        "API-Sign": get_kraken_signature("/0/private/GetWebSocketsToken", data, api_secret)
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['result']['token']
    else:
        raise Exception(f"Failed to get WebSocket token: {response.text}")


def on_message(ws, message):
    # print(type(message))
    # print(f"Received message: {message}")
    data = json.loads(message)
    print(data)

def on_error(ws, error):
    print(f"\n===========================================")
    print(f"===========Error: {error}=================")
    print(f"===========================================\n")

def on_close(ws, close_status_code, close_msg):
    print(f"\n===========================================")
    print("========WebSocket connection closed======")
    print(f"===========================================\n")
def on_open(ws):
    print("Kraken v2 Connection Opened.")
    
    # Authentication
    with open("/home/alg/.api.toml", "r") as fil:
        data = toml.load(fil)

    api_key = data['kraken_api']
    api_secret = data['kraken_sec']
    ws_token = get_ws_token(api_key, api_secret)
    
    # auth_message = {
        # "event": "subscribe",
        # "subscription": {
            # "name": "ownTrades",
            # "token": ws_token
        # }
    # }
    # ws.send(json.dumps(auth_message))
    
    # Subscribe to a public channel (e.g., ticker for BTC/USD)
    subscription = {
        "method": "subscribe",
        "params": {
            "channel": "level3",
            "symbol": ["BTC/USD"],
            "token": ws_token
        }
    }
    ws.send(json.dumps(subscription))

def run_websocket():
    websocket.enableTrace(True)
    conn = "wss://ws.kraken.com/v2"
    conn = "wss://ws-auth.kraken.com/v2"
    ws = websocket.WebSocketApp(conn,
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    
    ws.run_forever()


++Rcv raw: b'\x81~\x00\xf6{"channel":"level3","type":"update","data":[{"checksum":2689645963,"symbol":"BTC/USD","bids":[{"event":"add","order_id":"OAESWM-36V76-U5WCIA","limit_price":63111.0,"order_qty":3.96127153,"timestamp":"2024-09-24T05:20:37.190223016Z"}],"asks":[]}]}'


In [15]:

websocket_thread = threading.Thread(target=run_websocket)
websocket_thread.start()
try:
    websocket_thread.join()
except KeyboardInterrupt:
    print("Exiting...")


--- request header ---
GET /v2 HTTP/1.1
Upgrade: websocket
Host: ws-auth.kraken.com
Origin: https://ws-auth.kraken.com
Sec-WebSocket-Key: RPg0UR3vomvK1QRuFhGrdA==
Sec-WebSocket-Version: 13
Connection: Upgrade
Cookie: __cf_bm=aehmEqzh1Fuz1ssZs.w2gWGP9tQThh3ce.fewIyQMLI-1727154993-1.0.1.1-qzJCX3ecxJN1qukrTYrNFSL8qJ_bmm2vJdpWtw87bqBV570LccOa0KO9qYDVqgU5wsvHk7LZSlOvrywS1GYbRQ; _cfuvid=_sVor2rNjDN8qpG89I4BiB4ctl_a9J.NXx7ZDGWDOJc-1727154993009-0.0.1.1-604800000


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Tue, 24 Sep 2024 05:20:18 GMT
Connection: upgrade
upgrade: websocket
sec-websocket-accept: s3YFcf8S3vuwbYEQk1JUwVjee+s=
CF-Cache-Status: DYNAMIC
Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
X-Content-Type-Options: nosniff
Server: cloudflare
CF-RAY: 8c80530b7908799f-DEN
-----------------------
Websocket connected


Kraken v2 Connection Opened.


++Sent raw: b'\x81\xfe\x00\x87"N\x12\x93Yl\x7f\xf6V&}\xf7\x00t2\xb1Q;p\xe0A<{\xf1Gl>\xb3\x00>s\xe1C#a\xb1\x18ni\xb1A&s\xfdL+~\xb1\x18n0\xffG8w\xff\x11l>\xb3\x00=k\xfe@!~\xb1\x18nI\xb1`\x1aQ\xbcw\x1dV\xb1\x7fb2\xb1V!y\xf6Ll(\xb3\x00!^\xc4m=!\xf1\t(d\xdaj\x05D\xddL\x08y\xf5e{`\xf0d)^\xfeG\x18{\xea\x15-y\xa0t&k\xc6d\x1b}\xa7\x003o'
++Sent decoded: fin=1 opcode=1 data=b'{"method": "subscribe", "params": {"channel": "level3", "symbol": ["BTC/USD"], "token": "oLWOs3b+fvIHKVNnFkfG5rcFgLmeViy7ck3VhyUFUo4"}}'
++Rcv raw: b'\x81~\x00\x8b{"channel":"status","data":[{"api_version":"v2","connection_id":13469703312155409151,"system":"online","version":"2.0.8"}],"type":"update"}'
++Rcv decoded: fin=1 opcode=1 data=b'{"channel":"status","data":[{"api_version":"v2","connection_id":13469703312155409151,"system":"online","version":"2.0.8"}],"type":"update"}'
++Rcv raw: b'\x81~\x00\xb7{"method":"subscribe","result":{"channel":"level3","snapshot":true,"symbol":"BTC/USD"},"success":true,"time_in":"2024-09-24

{'channel': 'status', 'data': [{'api_version': 'v2', 'connection_id': 13469703312155409151, 'system': 'online', 'version': '2.0.8'}], 'type': 'update'}
{'method': 'subscribe', 'result': {'channel': 'level3', 'snapshot': True, 'symbol': 'BTC/USD'}, 'success': True, 'time_in': '2024-09-24T05:20:19.052512Z', 'time_out': '2024-09-24T05:20:19.059858Z'}


++Rcv raw: b'\x81~\x19\xd1{"channel":"level3","type":"snapshot","data": [{"symbol":"BTC/USD","checksum":4061653184,"bids":[{"order_id":"OL35PK-5U2PH-H3ZH3V","limit_price":63125.8,"order_qty":0.10000000,"timestamp":"2024-09-24T05:18:36.191436421Z"},{"order_id":"O727JF-3XQUX-NCE6EL","limit_price":63125.8,"order_qty":0.20000000,"timestamp":"2024-09-24T05:18:36.191674373Z"},{"order_id":"OFJD62-HPBDZ-T7IAVB","limit_price":63125.8,"order_qty":0.14603000,"timestamp":"2024-09-24T05:18:36.193209346Z"},{"order_id":"O7FA2F-3FFD2-KLF2V3","limit_price":63125.8,"order_qty":0.20000000,"timestamp":"2024-09-24T05:18:36.193577725Z"},{"order_id":"OXPFDJ-OWOMB-GXZJD6","limit_price":63125.8,"order_qty":0.40000000,"timestamp":"2024-09-24T05:18:36.193595265Z"},{"order_id":"OQIUDW-2NWLF-EIMWS5","limit_price":63125.8,"order_qty":0.20000000,"timestamp":"2024-09-24T05:18:36.193665930Z"},{"order_id":"O7M5D7-GSULO-K35NHE","limit_price":63125.8,"order_qty":0.90000000,"timestamp":"2024-09-24T05:18:37.420395591Z"},{"

...

{'channel': 'level3', 'type': 'update', 'data': [{'checksum': 1485561135, 'symbol': 'BTC/USD', 'bids': [{'event': 'delete', 'order_id': 'OXEGRU-5TB6B-HJ4BTP', 'limit_price': 63109.9, 'order_qty': 3.96134278, 'timestamp': '2024-09-24T05:20:40.413631239Z'}, {'event': 'add', 'order_id': 'OFX6B5-YLMJU-UJ5Z7I', 'limit_price': 63108.9, 'order_qty': 0.00015844, 'timestamp': '2024-09-24T05:11:18.870198008Z'}], 'asks': []}]}
{'channel': 'level3', 'type': 'update', 'data': [{'checksum': 254398673, 'symbol': 'BTC/USD', 'bids': [{'event': 'add', 'order_id': 'OTC2L3-TVO2S-KPOIVY', 'limit_price': 63112.1, 'order_qty': 3.96120036, 'timestamp': '2024-09-24T05:20:40.416540721Z'}], 'asks': []}]}
{'channel': 'level3', 'type': 'update', 'data': [{'checksum': 1485561135, 'symbol': 'BTC/USD', 'bids': [{'event': 'delete', 'order_id': 'OTC2L3-TVO2S-KPOIVY', 'limit_price': 63112.1, 'order_qty': 3.96120036, 'timestamp': '2024-09-24T05:20:40.440251435Z'}, {'event': 'add', 'order_id': 'OFX6B5-YLMJU-UJ5Z7I', 'limi

++Rcv raw: b'\x81\x17{"channel":"heartbeat"}'
++Rcv decoded: fin=1 opcode=1 data=b'{"channel":"heartbeat"}'
++Rcv raw: b'\x81~\x01\x84{"channel":"level3","type":"update","data":[{"checksum":2189629389,"symbol":"BTC/USD","bids":[],"asks":[{"event":"delete","order_id":"O7X4LD-NYRQA-YZO3UX","limit_price":63131.8,"order_qty":0.00633743,"timestamp":"2024-09-24T05:20:40.890279571Z"},{"event":"add","order_id":"OWVUT3-NE3TN-ZJNDAN","limit_price":63136.9,"order_qty":0.48283915,"timestamp":"2024-09-24T05:20:38.790976636Z"}]}]}'
++Rcv decoded: fin=1 opcode=1 data=b'{"channel":"level3","type":"update","data":[{"checksum":2189629389,"symbol":"BTC/USD","bids":[],"asks":[{"event":"delete","order_id":"O7X4LD-NYRQA-YZO3UX","limit_price":63131.8,"order_qty":0.00633743,"timestamp":"2024-09-24T05:20:40.890279571Z"},{"event":"add","order_id":"OWVUT3-NE3TN-ZJNDAN","limit_price":63136.9,"order_qty":0.48283915,"timestamp":"2024-09-24T05:20:38.790976636Z"}]}]}'


{'channel': 'heartbeat'}
{'channel': 'level3', 'type': 'update', 'data': [{'checksum': 2189629389, 'symbol': 'BTC/USD', 'bids': [], 'asks': [{'event': 'delete', 'order_id': 'O7X4LD-NYRQA-YZO3UX', 'limit_price': 63131.8, 'order_qty': 0.00633743, 'timestamp': '2024-09-24T05:20:40.890279571Z'}, {'event': 'add', 'order_id': 'OWVUT3-NE3TN-ZJNDAN', 'limit_price': 63136.9, 'order_qty': 0.48283915, 'timestamp': '2024-09-24T05:20:38.790976636Z'}]}]}


++Rcv raw: b'\x81~\x00\xf9{"channel":"level3","type":"update","data":[{"checksum":2105218953,"symbol":"BTC/USD","bids":[{"event":"modify","order_id":"OENJ7B-QAKHY-LSN2FA","limit_price":63125.8,"order_qty":0.00069827,"timestamp":"2024-09-24T05:20:12.907445945Z"}],"asks":[]}]}'
++Rcv decoded: fin=1 opcode=1 data=b'{"channel":"level3","type":"update","data":[{"checksum":2105218953,"symbol":"BTC/USD","bids":[{"event":"modify","order_id":"OENJ7B-QAKHY-LSN2FA","limit_price":63125.8,"order_qty":0.00069827,"timestamp":"2024-09-24T05:20:12.907445945Z"}],"asks":[]}]}'
++Rcv raw: b'\x81~\x00\xf9{"channel":"level3","type":"update","data":[{"checksum":1636604018,"symbol":"BTC/USD","bids":[{"event":"delete","order_id":"ORN6TP-6V337-FCFKGU","limit_price":63110.2,"order_qty":3.96131866,"timestamp":"2024-09-24T05:20:41.053042966Z"}],"asks":[]}]}'
++Rcv decoded: fin=1 opcode=1 data=b'{"channel":"level3","type":"update","data":[{"checksum":1636604018,"symbol":"BTC/USD","bids":[{"event":"delete","order_id

{'channel': 'level3', 'type': 'update', 'data': [{'checksum': 2105218953, 'symbol': 'BTC/USD', 'bids': [{'event': 'modify', 'order_id': 'OENJ7B-QAKHY-LSN2FA', 'limit_price': 63125.8, 'order_qty': 0.00069827, 'timestamp': '2024-09-24T05:20:12.907445945Z'}], 'asks': []}]}
{'channel': 'level3', 'type': 'update', 'data': [{'checksum': 1636604018, 'symbol': 'BTC/USD', 'bids': [{'event': 'delete', 'order_id': 'ORN6TP-6V337-FCFKGU', 'limit_price': 63110.2, 'order_qty': 3.96131866, 'timestamp': '2024-09-24T05:20:41.053042966Z'}], 'asks': []}]}
{'channel': 'level3', 'type': 'update', 'data': [{'checksum': 4180697916, 'symbol': 'BTC/USD', 'bids': [{'event': 'add', 'order_id': 'O24DXB-RFD6Q-M5GB5A', 'limit_price': 63112.2, 'order_qty': 3.96119789, 'timestamp': '2024-09-24T05:20:41.055838172Z'}], 'asks': []}]}
{'channel': 'level3', 'type': 'update', 'data': [{'checksum': 2431639741, 'symbol': 'BTC/USD', 'bids': [], 'asks': [{'event': 'add', 'order_id': 'OWHQQL-VLT4C-2GDXVQ', 'limit_price': 63131.8